In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import skimage.exposure
import numpy as np

In [ ]:
# Homomorphic filter class
class HomomorphicFilter:
    """Homomorphic filter implemented with diferents filters and an option to an external filter.
    
    High-frequency filters implemented:
        butterworth
        gaussian
    Attributes:
        a, b: Floats used on emphasis filter:
            H = a + b*H
        
        .
    """

    def __init__(self, a = 0.5, b = 1.5):
        self.a = float(a)
        self.b = float(b)

    # Filters
    def __butterworth_filter(self, I_shape, filter_params):
        P = I_shape[0]/2
        Q = I_shape[1]/2
        U, V = np.meshgrid(range(I_shape[0]), range(I_shape[1]), sparse=False, indexing='ij')
        Duv = (((U-P)**2+(V-Q)**2)).astype(float)
        H = 1/(1+(Duv/filter_params[0]**2)**filter_params[1])
        return (1 - H)

    def __gaussian_filter(self, I_shape, filter_params):
        P = I_shape[0]/2
        Q = I_shape[1]/2
        H = np.zeros(I_shape)
        U, V = np.meshgrid(range(I_shape[0]), range(I_shape[1]), sparse=False, indexing='ij')
        Duv = (((U-P)**2+(V-Q)**2)).astype(float)
        H = np.exp((-Duv/(2*(filter_params[0])**2)))
        return (1 - H)

    # Methods
    def __apply_filter(self, I, H):
        H = np.fft.fftshift(H)
        I_filtered = (self.a + self.b*H)*I
        return I_filtered

    def filter(self, I, filter_params, filter='butterworth', H = None):
        """
        Method to apply homormophic filter on an image
        Attributes:
            I: Single channel image
            filter_params: Parameters to be used on filters:
                butterworth:
                    filter_params[0]: Cutoff frequency 
                    filter_params[1]: Order of filter
                gaussian:
                    filter_params[0]: Cutoff frequency
            filter: Choose of the filter, options:
                butterworth
                gaussian
                external
            H: Used to pass external filter
        """

        #  Validating image
        if len(I.shape) != 2:
            raise Exception('Improper image')

        # Take the image to log domain and then to frequency domain 
        I_log = np.log1p(np.array(I, dtype="float"))
        I_fft = np.fft.fft2(I_log)

        # Filters
        if filter=='butterworth':
            H = self.__butterworth_filter(I_shape = I_fft.shape, filter_params = filter_params)
        elif filter=='gaussian':
            H = self.__gaussian_filter(I_shape = I_fft.shape, filter_params = filter_params)
        elif filter=='external':
            print('external')
            if len(H.shape) != 2:
                raise Exception('Invalid external filter')
        else:
            raise Exception('Selected filter not implemented')
        
        # Apply filter on frequency domain then take the image back to spatial domain
        I_fft_filt = self.__apply_filter(I = I_fft, H = H)
        I_filt = np.fft.ifft2(I_fft_filt)
        I = np.exp(np.real(I_filt))-1
        return np.uint8(I)

In [ ]:
def homo_filter(inputFile):
    # Main code
    img = cv2.imread(inputFile)[:,:,0]
    # lab = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    
    homo_filter = HomomorphicFilter(a = 0.5, b = 1.5)
    
    img = homo_filter.filter(I=img, filter_params=[3,3])
        

    
    # img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    return img

In [ ]:
def crop_left_and_right(inputFile, IMG_W=256, IMG_H=256):
    img = cv2.imread(inputFile)
    h = img.shape[0]
    w = img.shape[1]
    
    img_left = img[0:0+IMG_H, 0:0+IMG_W]
    img_right = img[h-IMG_H:h, w-IMG_W:w]
    
    return img_left, img_right

In [ ]:
def crop_left_and_right_tf(inputFile):
    ORI_SIZE = (271, 481)
    IMG_H = 271
    IMG_W = 256
    img = tf.io.read_file(inputFile)
    img = tf.io.decode_bmp(img, channels=3)
    # print(tf.rank(img))
    img = tf.cast(img, tf.float32)

    img_left = tf.image.crop_to_bounding_box(img, 0, 0, IMG_H, IMG_W)
    img_right = tf.image.crop_to_bounding_box(img, ORI_SIZE[0] - IMG_H, ORI_SIZE[1] - IMG_W, IMG_H, IMG_W)
    return img_left, img_right
    

In [ ]:
inputFile = "mura_data/RGB/old_best_mura/defect/BUTTERFLY (9).bmp"

defect_img = cv2.imread(inputFile)

plt.imshow(defect_img)

In [ ]:
# defect_img, grad_colored = contrast_depth_estimation(inputFile)
# defect_img = homo_filter(inputFile)
defect_img_left, defect_img_right = crop_left_and_right_tf(inputFile)
# defect_img = read_depth(inputFile)

plt.imshow(defect_img_left.numpy().astype(np.uint8))

In [ ]:
plt.imshow(defect_img_right.numpy().astype(np.uint8))

In [ ]:
inputFile = "mura_data/RGB/mura_clean/test_data/normal/normal.png"

normal_img = cv2.imread(inputFile)
# plt.imshow(normal_img)

In [ ]:
normal_img  = homo_filter(inputFile)
# normal_img, _  = contrast_depth_estimation(inputFile)
plt.imshow(normal_img.astype(np.uint8))